# Acordo de chaves para comunicação confidencial e autenticada

Construção de uma sessão síncrona de comunicação segura entre dois agentes (o Emitter e o Receiver), combinando os seguintes elementos constituintes:
- a cifra simétrica TAES (AES na sua versão tweak) usando autenticação do criptograma em cada superbloco; 
- o protocolo de acordo de chaves ECDH com verificação da chave, e autenticação dos agentes através do esquema de assinaturas ECDSA.

## Connection

In [6]:
from multiprocessing import Process, Pipe
from getpass import getpass

class Connection:
    def __init__(self, left, right, timeout=None):
        left_end, right_end = Pipe()
        self.timeout = timeout
        self.lproc = Process(target = left, args=(left_end,))
        self.rproc = Process(target = right, args=(right_end,))
        self.left = lambda : left(left_end)
        self.right = lambda : right(right_end)
        genDSAKeyPair("emitter")
        genDSAKeyPair("receiver")
        
    def auto(self, proc=None):
        if proc == None:
            self.lproc.start()
            self.rproc.start()
            self.lproc.join(self.timeout)
            self.rproc.join(self.timeout)
        else:
            proc.start()
            proc.join(self.timeout)
    def manual(self):
        self.left()
        self.right()

## Primitivas criptográficas

### Chaves de longa duração

A geração de chaves de longa duração é implementada pela função `genDSAKeyPair` que gera um par de chaves para assinaturas DSA e guarda cada uma em ficheiros **PEM**.
A chave privada é armazenada com recurso ao formato **PKCS8**.
Cada par de chaves é gerado apenas na criação da conexão **i.e.** chamada do construtor da classe `Connection`, mantendo-se constante para ligações estabelecidas na mesma instância da classe.

In [7]:
from cryptography.hazmat.primitives import serialization

def genDSAKeyPair(fname):
    privKeyF = open(fname+"_priv.pem", 'wb')
    pubKeyF = open(fname+"_pub.pem", 'wb')
   
    privKey = dsa.generate_private_key(
         key_size=2048,
         backend=default_backend()
    )
    pubKey = privKey.public_key()
    
    privKeyDump = privKey.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.BestAvailableEncryption(fname.encode("ascii")) # Store encrypted key
    )
    pubKeyDump = pubKey.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    privKeyF.write(privKeyDump)
    privKeyF.close()
    pubKeyF.write(pubKeyDump)
    pubKeyF.close()

O uso do esquema Encrypt-then-MAC permite garantir a autenticidade, integridade e confidencialidade das mensagens trocadas entre o `Emitter` e o `Receiver`.
Como tal, a cifra simétrica escolhida foi o AES (Advanced Encryption Standard) em modo CFB (Cipher Feedback Mode).
O MAC (Message Authentication Code) escolhido foi o HMAC (Hash-MAC) com SHA-256 que permite, por sua vez, garantir a integridade e autenticidade da mensagem trocada.

In [8]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.ciphers import algorithms, modes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.asymmetric import dsa
import os

# DH Parameters
P = 99494096650139337106186933977618513974146274831566768179581759037259788798151499814653951492724365471316253651463342255785311748602922458795201382445323499931625451272600173180136123245441204133515800495917242011863558721723303661523372572477211620144038809673692512025566673746993593384600667047373692203583
G = 44157404837960328768872680677686802650999163226766694797650810379076416463147265401084491113667624054557335394761604876882446924929840681990106974314935015501571333024773172440352475358750668213444607353872754650805031912866692119819377041901642732455911509867728218394542745330014071040326856846990119719675

def verifyHMAC(key, tag, ctxt):
    """ Computes the HMAC of a given cipher text and checkes whether it matches with
        the given tag i.e. validates the authenticity and integrity of the message
    """
    try:
        h = hmac.HMAC(key, hashes.SHA256(), backend=default_backend())
        h.update(ctxt)
        h.verify(tag)                        #exception on MAC failure
        return True
    except:
        return False

def extractMessage(key, tweak_h, ctxt):
    """ Extracts the plain text from a cryptogram resulting from a tweakable cipher
        where ptxt = dec(ctxt ^ hash(tweak)) ^ hash(tweak)
    """
    decryptor = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend()).decryptor()
    # ptxt = (decryptor.update(ctxt ^ tweak_h) + decryptor.finalize()) ^ tweak_h
    ptxt = decryptor.update(ctxt)
    return ptxt

def deriveKey(key_base, key_size, info):
    """ Expands key from key_base to key_size bytes
    """
    key = HKDF(hashes.SHA256(), length=key_size, salt=None, info=info, backend=default_backend()).derive(key_base)
    return key

def sign_message(priv_key, msg):
    """ Signs a particular message with a given private key
    """
    signature = priv_key.sign(msg,hashes.SHA256())
    return {'sig': signature, 'msg' : msg}

def verify_signature(pub_key, signature, msg):
    """ Verify that the private key associated with a particular public key was used to sign that particular message
    """
    try:
        pub_key.verify(signature,msg,hashes.SHA256())
        return True
    except:
        return False

## Emitter

In [12]:
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import (Cipher, algorithms, modes)
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.serialization import load_pem_private_key, load_pem_public_key
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
import os


def Emitter(conn):
    try:
        # Load long-term keys
        sign_key_dump = open("emitter_priv.pem", "rb").read()
        sign_key = load_pem_private_key(sign_key_dump, "emitter".encode("ascii"), backend=default_backend())
        
        ver_key_dump = open("receiver_pub.pem", "rb").read()
        ver_key = load_pem_public_key(ver_key_dump, backend=default_backend()) 
       
        # Session keys
        priv_key = ec.generate_private_key(ec.SECP521R1(), default_backend())
        public_key_pem = priv_key.public_key().public_bytes(
                                        encoding=serialization.Encoding.PEM,
                                        format=serialization.PublicFormat.SubjectPublicKeyInfo
                                        )
        
        payload = sign_message(sign_key, public_key_pem)
        conn.send(payload)                
        
        payload = conn.recv()    
        if verify_signature(ver_key, payload['sig'], payload['msg']) and payload['msg'] != b"0":
            g_y = load_pem_public_key(payload['msg'], backend=default_backend())
            # Generate shared secret
            s_key = priv_key.exchange(ec.ECDH(), g_y)

            #derive h_key and c_key from s_key 
            d_key = deriveKey(s_key, 32, b"ECDH Key Exchange")
            c_key = d_key[:16]
            h_key = d_key[16:]
            tweak = os.urandom(16)
            resp = b"1"
            
            while resp != b"0":
                header = 'TAES+HMAC_SHA256 Message'.encode("utf-8")
                content = '1234567890123456'

                cipher = Cipher(algorithms.AES(c_key),modes.ECB(),backend=default_backend()).encryptor()


                digest = hashes.Hash(hashes.SHA256(), backend = default_backend())
                digest.update(tweak)
                tweak_h = digest.finalize()

                #compute tweak hash (tweak_h = hash of tweak value)
                #content_new = (cipher.update(content.encode("utf-8") ^ tweak_h) + cipher.finalize())^tweak_h
                content_new = (cipher.update(content.encode("utf-8")) + cipher.finalize())
                
                #compute HMAC of header+content_new+iv+tweak_h and send in tag
                m_hmac = hmac.HMAC(h_key, hashes.SHA256(), backend = default_backend())
                m_hmac.update(header + content_new + tweak_h)
                tag = m_hmac.finalize()

                payload = {'header': header, 'content': content_new,'m_hmac': tag, 'tweak_h': tweak_h}
                conn.send(payload)
                # Ensure tweak is a nonce
                # tweak += 1
                resp = conn.recv()
        else:
            if payload['msg'] != b"0":
                sign = sign_message(sign_key, b"0")
                conn.send(sign)
            else:
                print("A verificação na troca de chaves falhou!")
        conn.close()
    except Exception as e:
        print("Emitter:" + e)
        print("Emitter: Error")

## Receiver

O Receiver, após a troca de chaves, e a cada mensagem recebida, efetua uma verificação da autenticidade e integridade da mesma com recurso à função `verifyHMAC` que recebe como parâmetro a chave usada no cálculo do MAC, o MAC esperado e o *input* sobre o qual este deve ser calculado.

Caso a mensagem recebida não tenha sido modificada, o `Receiver` procede ao processo de decifragem. Dado tratar-se de
uma mensagem cifrada com **TAES** é necessário manipular o criptograma de modo a obter a mensagem original, sendo útil ter em conta a seguinte expressão para o processo de decifragem:

$$ ctxt = E_k(ptxt \oplus h(tweak)) \oplus h(tweak)$$

onde:
- $ctxt$ : criptograma recebido
- $ptxt$ : mensagem original
- $tweak$: tweak/nonce

como tal, e tendo ainda em conta que $ D_k = E^{-1}_k $ , a obtenção do texto limpo original($ptxt$) pode ser conseguida da seguinte maneira:

$$ ptxt = D_k(ctxt \oplus h(tweak)) \oplus h(tweak) \rightarrow D_k(E_k(ptxt \oplus h(tweak)) \oplus h(tweak) \oplus h(tweak)) \oplus h(tweak) \rightarrow D_k(E_k(ptxt \oplus h(tweak))) \oplus h(tweak) \rightarrow ptxt \oplus h(tweak) \oplus h(tweak) \rightarrow ptxt $$

Este processo é implementado pela função `extractMessage` que recebe como parâmetros a chave acordada (`key`), o *hash* do *tweak* (`tweak_h`) e o criptograma (`ctxt`), devolvendo o resultado de aplicar a expressão anteriormente apresentada.
Dado ser utilizado em modo *tweaked*, é usado o modo ECB (*Electronic Code Book*) para a cifra AES.


In [19]:
def Receiver(conn):
    try:
        # Load long-term keys
        sign_key_dump = open("receiver_priv.pem", "rb").read()
        sign_key = load_pem_private_key(sign_key_dump, "receiver".encode("ascii"), backend = default_backend())
        
        ver_key_dump = open("emitter_pub.pem", "rb").read()
        ver_key = load_pem_public_key(ver_key_dump, backend = default_backend())

        payload = conn.recv()        
        if verify_signature(ver_key, payload['sig'], payload['msg']):
            # Session keys
            priv_key = ec.generate_private_key(ec.SECP521R1(), default_backend())
            public_key_pem = priv_key.public_key().public_bytes(
                                                encoding=serialization.Encoding.PEM,
                                                format=serialization.PublicFormat.SubjectPublicKeyInfo
                                                )
            g_x = load_pem_public_key(payload['msg'], backend=default_backend())

            payload = sign_message(sign_key, public_key_pem) 
            conn.send(payload) 

            s_key = priv_key.exchange(ec.ECDH(),g_x)
            d_key = deriveKey(s_key, 32, b"ECDH Key Exchange")
            c_key = d_key[:16]
            h_key = d_key[16:]
            payload = conn.recv()
            conn_alive = True
            
            if payload != b"0":
                while conn_alive:
                    header = payload['header']
                    ctxt = payload['content']
                    m_hmac = payload['m_hmac']
                    tweak_h = payload['tweak_h']

                    if verifyHMAC(h_key, m_hmac, header+ctxt+tweak_h):
                        msg = extractMessage(c_key, tweak_h, ctxt)
                        conn.send(msg)
                        print("Received: " + msg.decode("utf-8"))
                        conn_alive = (msg != b"0")
                    else:
                        conn.send(b"0")
                        # Close compromised connection
                        conn_alive = False
                    
                    if conn_alive:
                        payload = conn.recv()
            else:
                print("A verificação na troca de chaves falhou!")
        else:
            response = sign_message(sign_key,b"0")
            conn.send(response) 
        conn.close()
    except Exception as e:
        print("Receiver:" + e)
        print("Receiver: Erro na decifragem")

O estabelecimento de uma conexão entre os dois intervenientes (`Emitter` e `Receiver`)  é feita invocando o método `auto` da classe `Connection` que cria uma comunicação síncrona entre ambos. 
Este sincronismo é exigido pelo protocolo de troca de chaves (Diffie Hellman)

In [20]:
Connection(Emitter, Receiver, timeout=10).auto()

Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
R

Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
R

Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
R

Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
Received: 1234567890123456
R

Process Process-9:
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-12-485c6f803977>", line 66, in Emitter
    resp = conn.recv()
  File "/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk =

KeyboardInterrupt: 